In [ ]:
import json, os
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image
import torch
from research.utils.data_access_utils import S3AccessUtils, RDSAccessUtils
from research.weight_estimation.biomass_estimator import NormalizeCentered2D, NormalizedStabilityTransform, ToTensor, Network
from research.weight_estimation.weight_estimator import WeightEstimator
from research.weight_estimation.visualize import Visualizer
from research.weight_estimation.keypoint_utils.optics import euclidean_distance
from research.weight_estimation.keypoint_utils.keypoint_transformations import get_keypoint_arr, get_raw_3D_coordinates

<h1> Load Data </h1>

In [ ]:
s3_access_utils = S3AccessUtils('/root/data')
csv_url = 'https://aquabyte-calibrations.s3-eu-west-1.amazonaws.com/biomass_experiments/bolaks.pen88.matlab.02042020.cal.output.csv'
csv_f, bucket, key = s3_access_utils.download_from_url(csv_url)
df = pd.read_csv(csv_f)


In [ ]:
df['annotation'] = df.annotation.apply(lambda x: json.loads(x.replace("'", '"')))
df['camera_metadata'] = df.camera_metadata.apply(lambda x: json.loads(x.replace("'", '"')))
df['left_crop_metadata'] = df.left_crop_metadata.apply(lambda x: json.loads(x.replace("'", '"')))
df['right_crop_metadata'] = df.right_crop_metadata.apply(lambda x: json.loads(x.replace("'", '"')))



In [ ]:
df.to_csv('/root/data/alok/biomass_estimation/playground/bolaks_data.csv')

In [ ]:
s3_access_utils.s3_client.upload_file('/root/data/alok/biomass_estimation/playground/bolaks_data.h5', 
                            'aquabyte-images-adhoc', 'alok/bolaks_data.h5')

In [ ]:
df = pd.read_csv('/root/data/alok/biomass_estimation/playground/bolaks_data.csv')

In [ ]:
type(df.annotation.iloc[0])

In [ ]:
df.to_hdf('/root/data/alok/biomass_estimation/playground/bolaks_data.h5', 'table')

In [ ]:
s3_access_utils.download_from_url('https://aquabyte-images-adhoc.s3-eu-west-1.amazonaws.com/FishID_annotation_data/Precision_annotation/bolaks_results.zip')



In [ ]:
dirname = '/root/data/s3/aquabyte-images-adhoc/FishID_annotation_data/Precision_annotation/bolaks_results'
files = os.listdir(dirname)

In [ ]:
def write_list_to_file(guest_list, filename):
    """Write the list to csv file."""

    with open(filename, "w") as outfile:
        for entries in guest_list:
            outfile.write(entries)
            outfile.write("\n")

In [ ]:
write_list_to_file(sorted(files), '/root/data/alok/playground/output.csv')

In [ ]:
# instantiate weight estimator class

model_f = '/root/data/alok/biomass_estimation/playground/nn_epoch_253.pb'
weight_estimator = WeightEstimator(model_f)

# generate sample predictions
weights = []
for idx, row in df.iterrows():
    keypoints, camera_metadata = json.loads(row.annotation), json.loads(row.camera_metadata)
    weight_prediction = weight_estimator.predict(keypoints, camera_metadata)
    weights.append(weight_prediction)
    if len(weights) % 1000 == 0:
        print(len(weights))


In [ ]:
np.mean(weights)

In [ ]:
df.estimated_weight_g.mean()

In [ ]:
# initialize data transforms so that we can run inference with neural network
normalize_centered_2D_transform = NormalizeCentered2D()
normalized_stability_transform = NormalizedStabilityTransform()
to_tensor_transform = ToTensor()

# Get neural network weights from sample training
s3_access_utils = S3AccessUtils('/root/data')
model_url = 'https://aquabyte-models.s3-us-west-1.amazonaws.com/biomass/trained_models/2019-11-08T00-13-09/nn_epoch_798.pb'
model_f, _, _ = s3_access_utils.download_from_url(model_url)
network = torch.load(model_f)

weight_predictions = []
for idx, row in df.iterrows():
    input_sample = {
        'keypoints': json.loads(row.annotation),
        'cm': json.loads(row.camera_metadata),
        'stereo_pair_id': 0,
        'single_point_inference': True
    }
    nomralized_centered_2D_kps = \
        normalize_centered_2D_transform.__call__(input_sample)

    normalized_stability_kps = normalized_stability_transform.__call__(nomralized_centered_2D_kps)
    tensorized_kps = to_tensor_transform.__call__(normalized_stability_kps)
    weight_prediction = network(tensorized_kps['kp_input']).item() * 1e4
    weight_predictions.append(weight_prediction)


In [ ]:
df['new_weight'] = weights

In [ ]:
plt.figure(figsize=(20, 10))
plt.hist(df.new_weight.values - df.estimated_weight_g.values, bins=100)
plt.grid()
plt.show()

In [ ]:
def get_world_keypoints(row):
    return pixel2world(json.loads(row.annotation)['leftCrop'], 
                       json.loads(row.annotation)['rightCrop'], 
                       json.loads(row.camera_metadata))

def compute_yaw_angle(wkp):
    v = wkp['UPPER_LIP'] - wkp['TAIL_NOTCH']
    yaw_angle = np.arctan(v[1] / v[0]) * 180.0 / np.pi
    return yaw_angle

df['world_keypoints'] = df.apply(lambda x: get_world_keypoints(x), axis=1)
df['median_depth'] = df.world_keypoints.apply(lambda x: np.median([wkp[1] for wkp in x.values()]))
df['yaw_angle'] = df.world_keypoints.apply(lambda x: compute_yaw_angle(x))

In [ ]:
df['difference'] = df.new_weight - df.estimated_weight_g

In [ ]:
def display_crops(left_image, right_image, ann, overlay_keypoints=True, show_labels=False):
    fig, axes = plt.subplots(2, 1, figsize=(20, 20))
    axes[0].imshow(left_image)
    axes[1].imshow(right_image)
    
    left_keypoints = {item['keypointType']: [item['xCrop'], item['yCrop']] for item in ann['leftCrop']}
    right_keypoints = {item['keypointType']: [item['xCrop'], item['yCrop']] for item in ann['rightCrop']}
    if overlay_keypoints:
        for bp, kp in left_keypoints.items():
            axes[0].scatter([kp[0]], [kp[1]], color='red', s=1)
            if show_labels:
                axes[0].annotate(bp, (kp[0], kp[1]), color='red')
        for bp, kp in right_keypoints.items():
            axes[1].scatter([kp[0]], [kp[1]], color='red', s=1)
            if show_labels:
                axes[1].annotate(bp, (kp[0], kp[1]), color='red')
    plt.show()

In [ ]:
idx = 14
mask = (df.median_depth > 1.8) & (df.median_depth < 1.9)
tdf = df[mask].copy(deep=True)
left_crop_url = tdf.left_crop_url.iloc[idx]
right_crop_url = tdf.right_crop_url.iloc[idx]
print(tdf.difference.iloc[idx])
left_crop_f, _, left_crop_key = s3_access_utils.download_from_url(left_crop_url)
right_crop_f, _, right_crop_key = s3_access_utils.download_from_url(right_crop_url)
left_crop = Image.open(left_crop_f)
right_crop = Image.open(right_crop_f)

left_raw_image_key = os.path.join(os.path.dirname(left_crop_key), 'left_frame.resize_512_512.jpg')
left_raw_image_key = left_raw_image_key.replace('20200318.231820.Js_S', 'production')
right_raw_image_key = os.path.join(os.path.dirname(right_crop_key), 'right_frame.resize_512_512.jpg')
right_raw_image_key = right_raw_image_key.replace('20200318.231820.Js_S', 'production')
left_raw_image_f = s3_access_utils.download_from_s3('aquabyte-frames-resized-inbound', left_raw_image_key)
right_raw_image_f = s3_access_utils.download_from_s3('aquabyte-frames-resized-inbound', right_raw_image_key)

left_image = Image.open(left_crop_f)
right_image = Image.open(right_crop_f)
ann = json.loads(tdf.annotation.iloc[idx])
cm = json.loads(tdf.camera_metadata.iloc[idx])
display_crops(left_crop, right_crop, ann)

In [ ]:
Image.open(left_raw_image_f)

In [ ]:
Image.open(right_raw_image_f)

In [ ]:
get_keypoint_arr(ann, cm)

In [ ]:
depth_values = np.arange(0.5, 2.0, 0.1)
for idx in range(len(depth_values) - 1):
    low, high = depth_values[idx], depth_values[idx + 1]
    mean_difference = df[(df.median_depth > low) & (df.median_depth < high)].difference.median()
    print('Mean difference at depth range {}-{}: {}'.format(round(low, 1), round(high, 1), 
                                                            round(mean_difference, 2)))

In [ ]:
depth_values = np.arange(0.5, 2.0, 0.1)
for idx in range(len(depth_values) - 1):
    low, high = depth_values[idx], depth_values[idx + 1]
    std_difference = df[(df.median_depth > low) & (df.median_depth < high)].difference.std()
    print('Std difference at depth range {}-{}: {}'.format(round(low, 1), round(high, 1), 
                                                            round(std_difference, 2)))

In [ ]:
yaw_values = np.arange(0, 60, 10)
low_depth = 1.6
for idx in range(len(yaw_values) - 1):
    low, high = yaw_values[idx], yaw_values[idx + 1]
    mean_difference = df[(df.median_depth > low_depth) & (df.median_depth < low_depth + 0.1) & \
                         (df.yaw_angle.abs() > low) & (df.yaw_angle.abs() < high)].difference.mean()
    print('Mean difference at yaw range {}-{}: {}'.format(round(low, 1), round(high, 1), 
                                                            round(mean_difference, 2)))


In [ ]:
df[df.world_keypoints.apply(lambda x: x['TAIL_NOTCH'][0] < x['UPPER_LIP'][0])].difference.mean()

In [ ]:
df[df.world_keypoints.apply(lambda x: x['TAIL_NOTCH'][0] > x['UPPER_LIP'][0])].difference.mean()

In [ ]:
for idx, row in df[mask_x & mask_y].head(1).iterrows():
    input_sample = {
        'keypoints': row.annotation,
        'cm': row.camera_metadata,
        'stereo_pair_id': row.id,
        'single_point_inference': True
    }
    nomralized_centered_2D_kps = \
        normalize_centered_2D_transform.__call__(input_sample)
    normalized_stability_kps = normalized_stability_transform.__call__(nomralized_centered_2D_kps)
    tensorized_kps = to_tensor_transform.__call__(normalized_stability_kps)
    
    keypoint_arr = get_keypoint_arr(row.annotation, row.camera_metadata, recover_original_depth=True)
    
wkps = tensorized_kps['kp_input'].numpy()[0]
wkp_1 = (0.5 * 0.1) / wkps[:, 2]
x = wkps[:, 0] * wkp_1 / 0.5
y = wkps[:, 1] * wkp_1 / 0.5
plt.figure(figsize=(20, 10))
plt.scatter(x, y, color='blue')

x, y = keypoint_arr[:, 0], keypoint_arr[:, 2]
plt.scatter(x, y, color='red')

plt.grid()
plt.show()

In [ ]:
euclidean_distance(keypoint_arr[1], keypoint_arr[5])

In [ ]:
euclidean_distance(wkp['ANAL_FIN'], wkp['PELVIC_FIN'])

In [ ]:
def compute_medoid(wkp):
    return np.median(np.array([np.array(x) for x in wkp.values()]), axis=0)

df['medoid'] = df.world_keypoints.apply(lambda x: compute_medoid(x))

In [ ]:
mask_x = df.medoid.apply(lambda x: x[0]).abs() < 0.02
mask_y = df.medoid.apply(lambda x: x[2]).abs() < 0.02
df[mask_x & mask_y].difference.mean()

In [ ]:
df[mask_x & mask_y].estimated_weight_g.mean()

In [ ]:
df[mask_x & mask_y].new_weight.mean()

In [ ]:
mask = df.yaw_angle.abs() > 25
df[mask].difference.mean()

In [ ]:
from research.gtsf_data.body_parts import BodyParts

BODY_PARTS = BodyParts().get_core_body_parts()

def generate_rotation_matrix(n, theta):

    R = np.array([[
        np.cos(theta) + n[0] ** 2 * (1 - np.cos(theta)),
        n[0] * n[1] * (1 - np.cos(theta)) - n[2] * np.sin(theta),
        n[0] * n[2] * (1 - np.cos(theta)) + n[1] * np.sin(theta)
    ], [
        n[1] * n[0] * (1 - np.cos(theta)) + n[2] * np.sin(theta),
        np.cos(theta) + n[1] ** 2 * (1 - np.cos(theta)),
        n[1] * n[2] * (1 - np.cos(theta)) - n[0] * np.sin(theta),
    ], [
        n[2] * n[0] * (1 - np.cos(theta)) - n[1] * np.sin(theta),
        n[2] * n[1] * (1 - np.cos(theta)) + n[0] * np.sin(theta),
        np.cos(theta) + n[2] ** 2 * (1 - np.cos(theta))
    ]])

    return R

def normalize_3D_coordinates(wkps):

    v = np.median(wkps[:8], axis=0)
    v /= np.linalg.norm(v)
    y = np.array([0, 1, 0])
    n = np.cross(y, v)
    n /= np.linalg.norm(n)
    theta = -np.arccos(np.dot(y, v))
    R = generate_rotation_matrix(n, theta)
    wkps = np.dot(R, wkps.T).T

    # rotate about y-axis so that fish is straight
    upper_lip_idx = BODY_PARTS.index('UPPER_LIP')
    tail_notch_idx = BODY_PARTS.index('TAIL_NOTCH')
    v = wkps[upper_lip_idx] - wkps[tail_notch_idx]

    n = np.array([0, 1, 0])
    theta = np.arctan(v[2] / v[0])
    R = generate_rotation_matrix(n, theta)
    wkps = np.dot(R, wkps.T).T

    # perform reflecton if necessary
    tail_notch_idx = BODY_PARTS.index('TAIL_NOTCH')
    if wkps[upper_lip_idx][0] < wkps[tail_notch_idx][0]:
        R = np.array([
            [-1, 0, 0],
            [0, 1, 0],
            [0, 0, 1]
        ])
        wkps = np.dot(R, wkps.T).T

    return wkps

In [ ]:
idx = 5
raw_wkps = get_raw_3D_coordinates(df.annotation.iloc[idx], df.camera_metadata.iloc[idx])
norm_wkps, norm_wkps_2 = normalize_3D_coordinates(raw_wkps)
plt.scatter(norm_wkps[:, 0], norm_wkps[:, 2], color='blue')
plt.scatter(norm_wkps_2[:, 0], norm_wkps_2[:, 2], color='red')
plt.xlim(-0.4, 0.4)
plt.ylim(-0.4, 0.4)
plt.gca().set_aspect('equal', adjustable='box')
plt.grid()
plt.show()


In [ ]:
pdist(np.dot(raw_wkps, np.array([[1, 0, 0,], [0, 0, 0], [0, 0, 1]])))

In [ ]:
pdist(np.dot(norm_wkps, np.array([[1, 0, 0,], [0, 0, 0], [0, 0, 1]])))

In [ ]:
pdist(np.dot(norm_wkps_2, np.array([[1, 0, 0,], [0, 0, 0], [0, 0, 1]])))